In [1]:
import duckdb
import pandas as pd 
import os
from tqdm.auto import tqdm

In [2]:
%load_ext sql
conn = duckdb.connect('database.db')
%sql conn --alias duckdb

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [3]:
SEF = 1.0
ID = "Oranjestad_Aruba"
Name = "Oranjestad Aruba"
Lat = 12.52 #?
Lon = -70.04 #?
Alt = 5 #?
Source = "KNMI_Caribisch" #?
Link = "" #?
Vbl = "rr"
Stat = "sum"
Units = "mm"
Meta = "Data policy=open"

In [4]:
df = pd.DataFrame()

In [12]:
%%sql
df << SELECT timestamp, Precipitation, qPrecipitation FROM aruba.parquet

In [59]:
# Get timestamp as string:
df['timestamp'] = df['timestamp'].dt.tz_convert('UTC')
df_prep = df[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Precipitation', 'qPrecipitation']].copy()
df_prep['Year'] = df['timestamp'].dt.year.astype(str)
df_prep['Month'] = df['timestamp'].dt.month.astype(str).str.zfill(2)  # zero-padded
df_prep['Day'] = df['timestamp'].dt.day.astype(str).str.zfill(2)  # zero-padded
df_prep['Hour'] = df['timestamp'].dt.hour.astype(str).str.zfill(2)  # zero-padded
df_prep['Minute'] = df['timestamp'].dt.minute.astype(str).str.zfill(2)  # zero-padded
df_prep['Period'] = 'day'

# Add decimal:
df_prep['Precipitation'] = df['Precipitation']/10
df_prep['Meta'] = "qc=" + df_prep['qPrecipitation'].astype(str)

data_pd = df_prep[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Period', 'Precipitation', 'Meta']].copy()

In [60]:
SEF = 1.0
ID = "Oranjestad_Aruba"
Name = "Oranjestad Aruba"
Lat = 12.52 #?
Lon = -70.04 #?
Alt = 5 #?
Source = "KNMI_Caribisch" #?
Link = "" #?
Vbl = "rr"
Stat = "sum"
Units = "mm"
Meta = "Data policy=open"

sef_dict = {"SEF": SEF, "ID": ID, "Name": Name, "Lat": Lat, "Lon": Lon, "Alt": Alt, "Source": Source, 
            "Link": Link, "Vbl": Vbl, "Stat": Stat, "Units": Units, "Meta": Meta}

Year	Month	Day	Hour	Minute	Period	Value	Meta
1886	3	    18	12	    17	    0	    1005.14	orig=758.2mm|atb=24.8C|orig.time=8am

In [61]:
import copy
import codecs

def write_file(header: dict, data: pd.DataFrame, file_name: str):
    """Write the specified set of obs to a file in SEF format.

    Args:
        obs (:obj:`dict`): Dictionary
        file_name (:obj:`str`): File (or 'open'able object)

    Returns:
        :obj:`dict`: Data as key:value pairs.

    Raises:
        ValueError: obs not a SEF structure

    |
    """
    # Operate on local copy
    obs = copy.deepcopy(header)
    f = codecs.open(file_name, 'w', encoding='utf-8')
    # Meta might need packing
    # obs['Meta'] = _pack(obs['Meta'])
    # Header first
    for header in ('SEF', 'ID', 'Name', 'Source', 'Lat', 'Lon', 'Alt', 'Link', 'Vbl',
                   'Stat', 'Units', 'Meta'):
        if obs[header] is not None and obs[header] == obs[header]:
            f.write("%s\t%s\n" % (header, obs[header]))
        else:
            f.write("%s\t\n" % header)

    # Add the data table
    # obs['Data']['Meta'] = obs['Data']['Meta'].map(_pack, na_action='ignore')
    # print(obs['Data'])
    data.to_csv(f, sep='\t', header=True, index=False)

    # for i in range(1, len(obs['Data'])):
    #     obs['Data'][i].to_csv(f, sep='\t', header=False, index=False,)

In [62]:
write_file(sef_dict, data_pd, 'Oranjestad_Precip.tsv')